In [1]:
# Install the latest version of crossdeploy
# ! pip install --upgrade crossdeploy

# Node is installed by default in Watson Studio, JupyterLab and JupyterNotebook environments. 
# However, in case your environment does not have node installed, it can be installed with the below command.
# ! pip install "nodejs-bin[cmd]"

# Example 2: Build, promote and deploy a model

In this example, we will build, promote and deploy a simple SKLearn pipeline model.

# Imports

In [2]:
import pandas as pd

from crossdeploy.utils import utils
from crossdeploy.crossdeploy import CrossDeploy

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer

import ibm_watson_machine_learning

# Define variables

In [3]:
CLOUD_API_KEY = "xxx"
CLOUD_URL = "https://us-south.ml.cloud.ibm.com"
PROJECT_ID = "fc513094-926a-404b-9001-d15a03884123"
SPACE_ID = "9dd079b6-b503-4c77-ae3a-d10bf6d8adae"

wml_credentials = {"apikey": CLOUD_API_KEY, "url": CLOUD_URL}
wml_client = ibm_watson_machine_learning.APIClient(wml_credentials)

# Import data and preprocessing step

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/crossdeploy-io/crossdeploy-examples/main/data/mortgage-default.csv")

label_column = "MortgageDefault"
y = df[label_column]
X = df.drop(label_column, axis=1)

ct = make_column_transformer(
    (OneHotEncoder(), make_column_selector(dtype_include=object)),
    remainder="passthrough"
)

# Build model

In [5]:
model = Pipeline(steps=[
    ("transform", ct), 
    ("clf", RandomForestClassifier(n_estimators=3, max_depth=3, random_state=12345))
])
model.fit(X, y)
model.score(X, y)

0.639618138424821

# Save, promote and deploy model

Making changes, `MODEL_NAME` or `DEPLOYMENT_NAME`, will result in an update as there is no material impact to the underlying asset.

However, when the model is trained with new hyperparameters, this will result in destroy, the current version of the model and its promoted asset, creating and promoting a new model and updating the underlying asset of the existing deployment.

In [6]:
MODEL_NAME = "mortgage-model-rf"
DEPLOYMENT_NAME = "mortgage-model-rf"

flow = CrossDeploy()

flow.ibm.Provider(url=CLOUD_URL, api_key=CLOUD_API_KEY)

model_rf = flow.ibm.Model(model)(
    id = "mortgage-model-rf",
    name = MODEL_NAME,
    project_id = PROJECT_ID,
)

promoted_model_rf = flow.ibm.PromotedModel(model_rf)(
    id = "mortgage-model-rf-promoted",
    project_id = PROJECT_ID,
    space_id = SPACE_ID,
    asset_id = model_rf.id,
)

deployment = flow.ibm.Deployment(
    id = "mortgage-deployment-rf",
    name = DEPLOYMENT_NAME,
    space_id = SPACE_ID,
    asset = promoted_model_rf.id,
    online = True,
)

flow.apply()

deployment_state = utils.get_resource_by_name(deployment.friendly_unique_id)

print()
print(f"Deployment ID: {deployment_state['id']}")

Applying ...

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  + create

Terraform will perform the following actions:

  # ibmcpd_deployment.crossdeploy_mortgagedeploymentrf_58AD9F47 will be created
  + resource "ibmcpd_deployment" "crossdeploy_mortgagedeploymentrf_58AD9F47" {
      + asset    = (known after apply)
      + id       = (known after apply)
      + name     = "mortgage-model-rf"
      + online   = true
      + space_id = "9dd079b6-b503-4c77-ae3a-d10bf6d8adae"
      + url      = (known after apply)
    }

  # ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80 will be created
  + resource "ibmcpd_model" "crossdeploy_mortgagemodelrf_D0A9BA80" {
      + checksum      = "dfef315f632960684a95b017c32ea912"
      + id            = (known after apply)
      + model_path    = "dfef315f632960684a95b017c32ea912.tar.gz"
      + name          = "mortgage-model-rf"
      + project_id    = "fc51309

# Score model

In [7]:
wml_client.set.default_space(SPACE_ID)

deployment_state = utils.get_resource_by_name(deployment.friendly_unique_id)

deployment_id = deployment_state["id"]

payload = {"input_data": [{"values": X.head()}]}

wml_client.deployments.score(deployment_id, payload)


{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [['NO', [0.6, 0.39999999999999997]],
    ['NO', [0.5571018651362984, 0.44289813486370155]],
    ['NO', [0.5571018651362984, 0.44289813486370155]],
    ['YES', [0.49884053889123603, 0.5011594611087641]],
    ['NO', [0.6430252100840336, 0.3569747899159664]]]}]}

# Clean up

In [9]:
# flow.destroy()

Destroying ...
ibmcpd_model.crossdeploy_mortgagemodelrf_D0A9BA80: Refreshing state... [id=be406a11-bc06-4cce-a8bc-1e9a84a70a97]
ibmcpd_model.crossdeploy_mortgagemodelrfpromoted_9850AEA8: Refreshing state... [id=eabcdf7a-269e-47b3-b684-e9af09ed824f]
ibmcpd_deployment.crossdeploy_mortgagedeploymentrf_58AD9F47: Refreshing state... [id=17684c17-bd79-4d58-9fed-fef092e91620]

Terraform used the selected providers to generate the following execution
plan. Resource actions are indicated with the following symbols:
  - destroy

Terraform will perform the following actions:

  # ibmcpd_deployment.crossdeploy_mortgagedeploymentrf_58AD9F47 will be destroyed
  - resource "ibmcpd_deployment" "crossdeploy_mortgagedeploymentrf_58AD9F47" {
      - asset    = "eabcdf7a-269e-47b3-b684-e9af09ed824f" -> null
      - id       = "17684c17-bd79-4d58-9fed-fef092e91620" -> null
      - name     = "mortgage-model-rf" -> null
      - online   = true -> null
      - space_id = "9dd079b6-b503-4c77-ae3a-d10bf6d8adae